# <span style="color:teal;">CIS 211 Project 3:  OOP</span>

##### Due 11:00 P.M. April 21, 2017

##### Reading:  M&R 10.1 -- 10.4

Our goal this week is to gain some experience writing class definitions for some simple objects.  The classes are part of a solar system simulation.  This week we'll define the objects used in the simulation.  Next week we'll implement the simulation itself, and later in the term we'll add visualization. 

###  <span style="color:teal">Part 1: Vectors (50 points)</span>

Define a class named Vector.  Instances of this class will be Euclidan vectors (https://en.wikipedia.org/wiki/Euclidean_vector).  

An instance of this class will have three attributes, representing $x$, $y$, and $z$ coordinates in space.  When a Vector object is created the constructor should be passed three numbers to use as the initial values of the coordinates.

Your class should also have:
* accessor functions named `x`, `y`, and `z`, which return the current value of the specified coordinate
* a `__repr__` function that displays a Vector as a tuple of 3 numbers; use `"%.3g"` in the format statement so coordinates are shown with 3 significant digits
* methods that implement `==`, `+`, `-`, and `*` operations (see below)
* a method named `norm` that computes the length, or magnitude, of the vector, defined by the equation $$ \lVert \, v \, \rVert = \sqrt{x^2 + y^2 + z^2} $$
* a method named `clear` that sets all coordinates to 0

To see if two vectors are equal, simply see if their $x$, $y$, and $z$ components are the same.  If we simply compare floats this will almost certainly fail (due to roundoff errors) when the coordinates are of the size used in the solar system simulation, so your code should use the `round` builtin function to round coordinates to 3 decimal places.

To add or subtract two vectors, make a new vector that has the sum or difference of the components of two existing vectors.

The method that implemements multiplication should implement scalar multiplication: the second operand should be an integer or a float, and the result is a new vector where all components are multiplied by the scalar.

Example:
<pre>
>>> v1 = Vector(3, 5, 0)
>>> v2 = Vector(1, 1, 4)

>>> v1.x()
3

>>> v1.y()
5

>>> v1.norm()
5.830951894845301

>>> v1 + v2
(4,6,4)

>>> v1 - v2
(2,4,-4)

>>> v2 * 3
(3,3,12)

>>> v1 + v2 == Vector(4,6,4)
True

>>> v1.clear()
>>> v1
(0,0,0)
</pre>

##### <span style="color:red">Code:</span>

In [71]:
from math import sqrt
from operator import *


class Vector:
    """
    A Vector is a 3-tuple of (x,y,z) coordinates.
    """
    def __init__(self,x,y,z):
        self._x = x
        self._y = y
        self._z = z
    
    def x(self):
        return self._x
    
    def y(self):
        return self._y
    
    def z(self):
        return self._z
    
    
    def __repr__(self):
        return ('({:.3g},{:.3g},{:.3g})'.format((self.x()), self.y(), self.z()))
    
    def __eq__(self, other):
        if (self.x() == other.x() and self.y() == other.y() and self.z() == other.z()):
            return True
        else:
            return False
            
    
    def __add__(self, other):
        return Vector((self._x + other._x), (self._y + other._y), (self._z + other._z))
    
    def __sub__(self, other):
        return Vector((self._x - other._x), (self._y - other._y), (self._z - other._z))
    
    
    def __mul__(self, other):
        return Vector((self._x * other), (self._y * other), (self._z * other))
    
    
    def norm(self):
        return sqrt((self._x*self._x) + (self._y*self._y) + (self._z*self._z))

    def clear(self):
        self._x = 0
        self._y = 0
        self._z = 0
    
    



##### <span style="color:red">Autograder Tests:</span>

In [72]:
v1 = Vector(1, 1, 1)
v2 = Vector(2, 2, 2)
v3 = Vector(3, 3, 3)

In [73]:
assert str(v1) == '(1,1,1)'
assert str(v2) == '(2,2,2)'

In [74]:
assert Vector(1, 2, 3) == Vector(1, 2, 3)
assert Vector(1, 2, 3) != Vector(1.001, 2.001, 3.001)

In [75]:
assert v1 + v2 == v3

In [76]:
assert v3 - v2 == v1

In [77]:
assert v3 == v1 * 3

In [78]:
assert round(v2.norm(), 10) == round(sqrt(12), 10)

In [79]:
v1.clear()
assert v1 == Vector(0,0,0)

##### <span style="color:red">Documentation:</span>

The point of this class is to allow you to put together a Vector with 3 coordinates and it also allows you to add vectors, subtract, and multiply vecotrs together. Another part of this vector class is it allows you to find the magnitude of the vector with the function, norm. 

###  <span style="color:teal">Part 2: Celestial Bodies (50 points)</span>

Define a class named `Body` that can be used to represent objects in an $n$-body simulation, _e.g._ the sun and planets in the solar system.

Each instance of this class should have the following attributes:
* mass, represented by a single floating point number
* position, velocity, and force, each of which is a 3D vector

Define the constructor so that all attributes are optional.  The default mass is 0, and position and velocity are (0,0,0).  The force vector should always be initialized to (0,0,0).

The representation string for a Body should include the mass, position, and velocity:

<pre>
>>> Body()
0kg (0,0,0) (0,0,0)

>>> ep = Vector(-2.700743E+10, 1.446007E+11, 9686451)
>>> Body(mass = 5.9736E+24, position = ep)
5.97e+24kg (-2.7e+10,1.45e+11,9.69e+06) (0,0,0)
</pre>

Your class should also include the following:
* accessor functions called `mass`, `position`, `velocity`, and `force` that return the corresponding attribute
* a method called `direction` which takes another Body object as a parameter and returns a vector that "points at" the other Body (see example below)
* a method named `clear_force` that sets the force vector to (0,0,0) by calling that vector's `clear` method
* an `add_force` method that takes another Body as a parameter and updates the force vector using the equation shown below
* a method called `move` that will use the current value of the force vector to update the object's position using the algorithm outlined below.

This example illustrates the `direction` method.  Notice how the direction from `b1` to `b2` is the same size but points the other way from the direction from `b2` to `b1`.
<pre>
>>> b1 = Body(position = Vector(0,1,0))
>>> b2 = Body(position = Vector(1,0,0))

>>> b1.direction(b2)
(1,-1,0)

>>> b2.direction(b1)
(-1,1,0)
</pre>

To implement `add_force`, first compute the force pulling a body A toward another body B. Let $\vec{d}$ be the direction from A to B. Then a vector $\vec{f}$ that defines the force is

$$
\vec{f} = \frac{\vec{d} \times m_\mathrm{B}}{{\lVert \, \vec{d} \, \rVert}^3}
$$

where $m_\mathrm{B}$ is the mass of body B.  Add $\vec{f}$ to the force vector in the object that represents A.

The `move` method will take a parameter named `dt` which is a time step size.
To move a body A use the accumulated forces created by previous calls to `add_force`.
First compute a vector that represents the acceleration of A:

$$
\vec{a} = G \times \vec{f}_\mathrm{A} 
$$

where $G$ is the universal gravitational constant and $\vec{f}_\mathrm{A}$ is A's force vector.

Then update A's velocity vector:
$
\vec{v}_\mathrm{A} = \vec{v}_\mathrm{A} + \vec{a} \times \mathtt{dt}
$

Finally, update A's position vector:
$
\vec{p}_\mathrm{A} =  \vec{p}_\mathrm{A} + \vec{v}_\mathrm{A} \times \mathtt{dt}
$



##### <span style="color:red">Code:</span>

Put your class definition in the following code cell.  Note the gravitational constant $G$ has been defined for you.

In [107]:
G = 6.67E-11

class Body:
    """
    A Body object represents the state of a celestial body.  A body has mass 
    (a scalar), position (a vector), and velocity (a vector).  A third vector, 
    named force, is used when calculating forces acting on a body.
    """
    def __init__(self, mass = 0, position = Vector(0,0,0), velocity = Vector(0,0,0)):
        self._mass = mass
        self._pos = position
        self._vel = velocity
        self._force = Vector(0,0,0)
        
    def __repr__(self):
        return ("{:.3g}kg {} {}".format(self.mass(), self.position(), self.velocity()))
    
    def mass(self):
        return self._mass
    
    def position(self):
        return self._pos
    
    def velocity(self):
        return self._vel
    
    def force(self):
        return self._force
    
    def direction(self, other):
        return other.position() - self.position()
    
    def clear_force(self):
        self.force().clear()
        
    def add_force(self, other):
        self._force += self.direction(other) * (other.mass() / self.direction(other).norm() ** 3)
        print (self._force)
        



##### <span style="color:red">Tests:</span>

**Optional** &nbsp; If you want to do your own tests use the code cell below to create and test objects.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

In [108]:
melon = Body(mass=3.0, position=Vector(0,6371000,0))
earth = Body(mass=5.9736E+24)

In [109]:
assert str(melon) == "3kg (0,6.37e+06,0) (0,0,0)"
assert str(earth) == "5.97e+24kg (0,0,0) (0,0,0)"

In [110]:
assert earth.direction(melon) == Vector(0,6371000,0)
assert melon.direction(earth) == Vector(0,-6371000,0)

In [111]:
melon.add_force(earth)
assert -1.48e+11 < melon.force().y() < -1.47e+11

(0,-1.47e+11,0)


In [112]:
melon.move(1)
assert 6370990 < melon.position().y() < 6371000

AttributeError: 'Body' object has no attribute 'move'

In [113]:
melon.clear_force()
assert melon.force() == Vector(0,0,0)

##### <span style="color:red">Documentation:</span>

The point of this class is to allow you to use the body as if they were planets in the solar system and it allows you to find the magnitude, velocity, force, and basically the movements of the celestial body. 